# Handling Missing Values

#### Let's get out dataset and important libraries

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.impute import KNNImputer

In [24]:
# save filepath to variable for easier access
melbourne_file_path = '../hitchhikersGuideToMachineLearning/home-data-for-ml-course/train.csv'
# read the data and store data in DataFrame titled melbourne_data
train_data = pd.read_csv(melbourne_file_path) 
# print a summary of the data in Melbourne data
train_data.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


You can already see a few missing values in the first several rows.  In the next step, you'll obtain a more comprehensive understanding of the missing values in the dataset.

# Step 1: Preliminary investigation

Run the code cell below without changes.

In [25]:
# Shape of training data (num_rows, num_columns)
print(train_data.shape)

(1460, 81)


In [26]:
# Number of missing values in each column of training data
missing_val_count_by_column = (train_data.isnull().sum())
missing_val_count_by_column[missing_val_count_by_column > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

Here is another piece of code used for same but with percentages too.

In [27]:
def missingcheck(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
    return missing_data

In [28]:
#just pass the data to the function
missingcheck(train_data)

,Total,%
PoolQC,1453,99.5
MiscFeature,1406,96.3
Alley,1369,93.8
Fence,1179,80.8
FireplaceQu,690,47.3
...,...,...
CentralAir,0,0.0
SaleCondition,0,0.0
Heating,0,0.0
TotalBsmtSF,0,0.0


### Also seggregate Numerical attributes and Categorical attributes
Becuase it will affect the strategy we will follow for missing value treatment!
#### Numerical data:
These data have meaning as a measurement, such as a person’s height, weight, IQ, or blood pressure; or they’re a count, such as the number of stock shares a person owns,how many teeth a dog has, or how many pages you can read of your favorite book before you fall asleep.

#### Categorical data: 
Categorical data represent characteristics such as a person’s gender, marital status, hometown, or the types of movies they like. Categorical data can take on numerical values (such as “1” indicating male and “2” indicating female), but those numbers don’t have mathematical meaning

In [29]:
#Numerical data will generally have int64 or float64 representation while 
#Categorical may follow object datatype.

train_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

You can treat missing values seperately and later join them together! But to keep this notebook short and understandable I will continue only with numerical values.

In [30]:
train_data = train_data.select_dtypes(exclude=['object'])
#train_data_Categorical = train_data.select_dtypes(include=['object'])


Select target variable and then Split the data in training and validation set

In [31]:
X = train_data.drop(['SalePrice'] , axis =1)
y = train_data.SalePrice

In [32]:
#Splitting in training and Validation set
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

To compare different approaches to dealing with missing values, you'll use the same `score_dataset()` function from the tutorial.  This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### We are all set to LAUNCH!

# Step 2: Apply different Techniques



## 1. Drop Missing Values
In this step, you'll preprocess the data in `X_train` and `X_valid` to remove columns with missing values.  Set the preprocessed DataFrames to `reduced_X_train` and `reduced_X_valid`, respectively.  

In [34]:
#get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
# drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)


If missing values are very less, you can also drop the whole row containing it.

Run the next code cell without changes to obtain the MAE for this approach.

In [35]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17952.591404109586


## 2. Univariate Imputation
###### Fancy words for replacinig missing values with mean, median, mode or some constant value!

One type of imputation algorithm is univariate, which imputes values in the i-th feature dimension using only non-missing values in that feature dimension (e.g. impute.SimpleImputer).


Use the next code cell to impute missing values with the mean value along each column.  Set the preprocessed DataFrames to `imputed_X_train` and `imputed_X_valid`.  Make sure that the column names match those in `X_train` and `X_valid`.

In [36]:
from sklearn.impute import SimpleImputer

# imputation
my_imputer = SimpleImputer()

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))

imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))


# imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


One can use Mean, Median, Mode or Constant value to replace the missing data. Remeber I told you that our strategy for numerical and categorical attribute will be different. Observe yourself, if coloumn have categorical data like high, medium,low one can not use mean.

Run the next code cell without changes to obtain the MAE for this approach.

In [37]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18250.608013698627


## 3. Multivariate Imputation
###### Fancy words for predicting  missing values with non-missing subset of data!

Multivariate imputation algorithms use the entire set of available feature dimensions to estimate the missing values (e.g. impute.IterativeImputer).

In [38]:
my_imputer = IterativeImputer(BayesianRidge())

# 
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))

imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))


# imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


In [39]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
17976.146438356165


##### Other estimators we can use 
#### BayesianRidge: regularized linear regression - default

#### DecisionTreeRegressor: non-linear regression

#### ExtraTreesRegressor: similar to missForest in R

#### KNeighborsRegressor: comparable to other KNN imputation approaches

## 4. Nearest neighbors imputation
###### Fancy word for KNN to select constant value for imputation
NOTE: IterativeImputer KNeighbourNRegressor is different from this. 

In [40]:
my_imputer = KNNImputer(n_neighbors=2, weights="uniform")


imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))

imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))


# imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [41]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18095.66794520548


 ## 5. An Extension To Imputation
Imputation is the standard approach, and it usually works well. However, imputed values may by systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing. Here's how it might look:

In [42]:
imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_valid.copy()

cols_with_missing = (col for col in X_train.columns 
                                 if X_train[col].isnull().any())
for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()
#see what happend to the dataset
imputed_X_train_plus.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,LotFrontage_was_missing,MasVnrArea_was_missing,GarageYrBlt_was_missing
618,619,20,90.0,11694,9,5,2007,2007,452.0,48,...,0,0,260,0,0,7,2007,False,False,False
870,871,20,60.0,6600,5,5,1962,1962,0.0,0,...,0,0,0,0,0,8,2009,False,False,False
92,93,30,80.0,13360,5,7,1921,2006,0.0,713,...,44,0,0,0,0,8,2009,False,False,False
817,818,20,NaN,13265,8,5,2002,2002,148.0,1218,...,0,0,0,0,0,7,2008,True,False,False
302,303,20,118.0,13704,7,5,2001,2002,150.0,0,...,0,0,0,0,0,1,2006,False,False,False


In [43]:
# And now we Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus)
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)



Run the next code cell without changes to obtain the MAE for this approach.

In [44]:
print("Mean Absolute Error from Imputation while Track What Was Imputed:")
print(score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_valid))

Mean Absolute Error from Imputation while Track What Was Imputed:
18253.31479452055


### This whole can be also very easily done by Marking imputed values
The MissingIndicator transformer is useful to transform a dataset into corresponding binary matrix indicating the presence of missing values in the dataset. 

## Step 3: Devising Ultimate Strategy 

### Comparing All Solutions
It is discussed in the article [link]
